# Обработка датасета

## Библиотеки и скрипты

In [1]:
import numpy as np
import pandas as pd
import re
from transliterate import translit, get_available_language_codes

# Библиотека для предобработки датасетов
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from data_preprocessing import DataPreprocessor
dp = DataPreprocessor()

# Отключаем предупреждения
import warnings
warnings.simplefilter('ignore')

## Обзор данных

In [2]:
# Загружаем датасеты без 1-й строки и делаем переиндексацию с удалением столбца index
df1 = pd.read_csv('../data/hubex.ru-ClientID-Sources.csv')[1:].reset_index(drop=True) # Источники
df2 = pd.read_csv('../data/hubex.ru-ClientID-Behavior.csv')[1:].reset_index(drop=True) # Поведение
# Удаляем дубликаты по ClientID
df1 = df1.drop_duplicates(subset='ClientID', keep="first")
df2 = df2.drop_duplicates(subset='ClientID', keep="first")

In [3]:
df1.head(5)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Дата визита,Визиты,Посетители,Отказы,Глубина просмотра,Время на сайте,Новые посетители,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели
0,15785624736522097,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,2020-01-11,15,1,0.066667,3.800000,00:21:41,0,0.000000,0.933333,0.066667,57
1,158240803283368197,Прямые заходы,Не определено,Не определено,Не определено,Не определено,2020-02-23,15,15,1.000000,1.000000,00:00:00,15,NaN,0.000000,0.000000,4
2,1594782414605357519,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,2020-07-21,14,1,1.000000,1.000000,00:00:00,0,0.357143,0.928571,0.071429,0
3,1595636421722944149,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,2020-07-25,14,1,0.571429,1.928571,00:00:00,1,0.000000,0.928571,0.000000,0
4,1600595766306691164,Переходы по рекламе,Не определено,Не определено,yad,300x300,2020-09-20,11,1,0.181818,1.000000,00:00:10,1,0.000000,0.909091,0.000000,0


In [4]:
df1.shape

(58888, 17)

In [5]:
df2.head(5)

,ClientID,Дата первого визита,Город,Достигнутая цель,Путь (полный) страницы входа,Путь (полный) страницы выхода,Визиты,Мобильность
0,1577519126226205712,2019-12-28,Москва,Не определено,/features/,/features/,165,0.0
1,1544792334967355948,2018-12-14,Санкт-Петербург,Не определено,/,/,162,0.0
2,1564737646183903772,2019-08-02,Санкт-Петербург,Не определено,/price/,/price/,103,0.0
3,1599476927749455055,2020-09-07,Санкт-Петербург,Hubex.ru (Ретаргетинг.Все-Страницы) - Посетил ...,/,/,80,0.0
4,1589375617652616197,2020-05-13,Санкт-Петербург,Не определено,/,/,63,0.0


In [6]:
df2.shape

(57730, 8)

## Обработка данных

In [7]:
# Производим слияние датасетов
df = pd.merge(df1, df2, how='left')

In [8]:
df.head(5)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Дата визита,Визиты,Посетители,Отказы,...,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели,Дата первого визита,Город,Достигнутая цель,Путь (полный) страницы входа,Путь (полный) страницы выхода,Мобильность
0,15785624736522097,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,2020-01-11,15,1,0.066667,...,0.000000,0.933333,0.066667,57,NaN,NaN,NaN,NaN,NaN,NaN
1,158240803283368197,Прямые заходы,Не определено,Не определено,Не определено,Не определено,2020-02-23,15,15,1.000000,...,NaN,0.000000,0.000000,4,NaN,NaN,NaN,NaN,NaN,NaN
2,1594782414605357519,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,2020-07-21,14,1,1.000000,...,0.357143,0.928571,0.071429,0,NaN,NaN,NaN,NaN,NaN,NaN
3,1595636421722944149,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,2020-07-25,14,1,0.571429,...,0.000000,0.928571,0.000000,0,2020-07-25,Нур-Султан,Не определено,/#secondPage,/#secondPage,1.0
4,1600595766306691164,Переходы по рекламе,Не определено,Не определено,yad,300x300,2020-09-20,11,1,0.181818,...,0.000000,0.909091,0.000000,0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.shape

(58888, 23)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58888 entries, 0 to 58887
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ClientID                       58888 non-null  object 
 1   Первый источник трафика        58888 non-null  object 
 2   Первый переход с сайтов        58888 non-null  object 
 3   Первая поисковая система       58888 non-null  object 
 4   UTM Source                     58888 non-null  object 
 5   UTM Content                    58888 non-null  object 
 6   Дата визита                    58888 non-null  object 
 7   Визиты                         58888 non-null  int64  
 8   Посетители                     58888 non-null  int64  
 9   Отказы                         58888 non-null  float64
 10  Глубина просмотра              58888 non-null  float64
 11  Время на сайте                 58888 non-null  object 
 12  Новые посетители               58888 non-null 

In [11]:
df.columns

Index(['ClientID', 'Первый источник трафика', 'Первый переход с сайтов',
       'Первая поисковая система', 'UTM Source', 'UTM Content', 'Дата визита',
       'Визиты', 'Посетители', 'Отказы', 'Глубина просмотра', 'Время на сайте',
       'Новые посетители', 'Дней между визитами', 'Вернувшиеся: 1 день',
       'Вернувшиеся: 2‑7 дней', 'Достижения любой цели', 'Дата первого визита',
       'Город', 'Достигнутая цель', 'Путь (полный) страницы входа',
       'Путь (полный) страницы выхода', 'Мобильность'],
      dtype='object')

In [13]:
# Удаление ненужных данных
columns = ['Посетители']
df = dp.drop_col(df, columns)

# Сегментация пользователей
df.loc[df['Дата визита'].isnull(), 'Дата визита'] = 'undefined'
# df = df[df['Дата визита'].str.contains('202.*')]
# df = df.loc[(df['Визиты'] <= 35) & (df['Достижения любой цели'] <= 100)]
# df = df.loc[(df['ClientID'] != '')]

df.head(2)

,ClientID,Первый источник трафика,Первый переход с сайтов,Первая поисковая система,UTM Source,UTM Content,Дата визита,Визиты,Отказы,Глубина просмотра,...,Дней между визитами,Вернувшиеся: 1 день,Вернувшиеся: 2‑7 дней,Достижения любой цели,Дата первого визита,Город,Достигнутая цель,Путь (полный) страницы входа,Путь (полный) страницы выхода,Мобильность
0,15785624736522097,Переходы по рекламе,Не определено,Не определено,Не определено,Не определено,2020-01-11,15,0.066667,3.8,...,0.0,0.933333,0.066667,57,NaN,NaN,NaN,NaN,NaN,NaN
1,158240803283368197,Прямые заходы,Не определено,Не определено,Не определено,Не определено,2020-02-23,15,1.000000,1.0,...,NaN,0.000000,0.000000,4,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Переименование столбцов
df.columns = dp.get_translite(df.columns, 'ym_')
df.columns

Index(['ym_clientid', 'ym_pervyj_istochnik_trafika',
       'ym_pervyj_perehod_s_sajtov', 'ym_pervaja_poiskovaja_sistema',
       'ym_utm_source', 'ym_utm_content', 'ym_data_vizita', 'ym_vizity',
       'ym_otkazy', 'ym_glubina_prosmotra', 'ym_vremja_na_sajte',
       'ym_novye_posetiteli', 'ym_dnej_mezhdu_vizitami',
       'ym_vernuvshiesja_1_den', 'ym_vernuvshiesja_2_7_dnej',
       'ym_dostizhenija_ljuboj_tseli', 'ym_data_pervogo_vizita', 'ym_gorod',
       'ym_dostignutaja_tsel', 'ym_put_polnyj_stranitsy_vhoda',
       'ym_put_polnyj_stranitsy_vyhoda', 'ym_mobilnost'],
      dtype='object')

**Обработка пропусков**

In [15]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58888 entries, 0 to 58887
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ym_clientid                     58888 non-null  object 
 1   ym_pervyj_istochnik_trafika     58888 non-null  object 
 2   ym_pervyj_perehod_s_sajtov      58888 non-null  object 
 3   ym_pervaja_poiskovaja_sistema   58888 non-null  object 
 4   ym_utm_source                   58888 non-null  object 
 5   ym_utm_content                  58888 non-null  object 
 6   ym_data_vizita                  58888 non-null  object 
 7   ym_vizity                       58888 non-null  int64  
 8   ym_otkazy                       58888 non-null  float64
 9   ym_glubina_prosmotra            58888 non-null  float64
 10  ym_vremja_na_sajte              58888 non-null  object 
 11  ym_novye_posetiteli             58888 non-null  int64  
 12  ym_dnej_mezhdu_vizitami         

ym_clientid                           0
ym_pervyj_istochnik_trafika           0
ym_pervyj_perehod_s_sajtov            0
ym_pervaja_poiskovaja_sistema         0
ym_utm_source                         0
ym_utm_content                        0
ym_data_vizita                        0
ym_vizity                             0
ym_otkazy                             0
ym_glubina_prosmotra                  0
ym_vremja_na_sajte                    0
ym_novye_posetiteli                   0
ym_dnej_mezhdu_vizitami           51740
ym_vernuvshiesja_1_den                0
ym_vernuvshiesja_2_7_dnej             0
ym_dostizhenija_ljuboj_tseli          0
ym_data_pervogo_vizita            30774
ym_gorod                          30774
ym_dostignutaja_tsel              30774
ym_put_polnyj_stranitsy_vhoda     30774
ym_put_polnyj_stranitsy_vyhoda    30774
ym_mobilnost                      30774
dtype: int64

In [16]:
# Корректировка значений
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Не определено', 'ym_pervyj_istochnik_trafika'] = 'undefined'
# ------ Для склейки заменяем пропуски на np.nan
df.loc[df['ym_pervyj_perehod_s_sajtov'] == 'Не определено', 'ym_pervyj_perehod_s_sajtov'] = np.nan
df.loc[df['ym_pervaja_poiskovaja_sistema'] == 'Не определено', 'ym_pervaja_poiskovaja_sistema'] = np.nan
df.loc[df['ym_utm_source'] == 'Не определено', 'ym_utm_source'] = np.nan
df.loc[df['ym_utm_content'] == 'Не определено', 'ym_utm_content'] = np.nan
# ------
df.loc[df['ym_gorod'].isnull(), 'ym_gorod'] = 'undefined'
df.loc[df['ym_gorod'] == 'Не определено', 'ym_gorod'] = 'undefined'
df.loc[df['ym_otkazy'].isnull(), 'ym_otkazy'] = df['ym_otkazy'].median()
df.loc[df['ym_glubina_prosmotra'].isnull(), 'ym_glubina_prosmotra'] = df['ym_glubina_prosmotra'].median()
df.loc[df['ym_vremja_na_sajte'].isnull(), 'ym_vremja_na_sajte'] = 'undefined'
df.loc[df['ym_novye_posetiteli'].isnull(), 'ym_novye_posetiteli'] = 'undefined'
df.loc[df['ym_dnej_mezhdu_vizitami'].isnull(), 'ym_dnej_mezhdu_vizitami'] = df['ym_dnej_mezhdu_vizitami'].median()
df.loc[df['ym_vernuvshiesja_1_den'].isnull(), 'ym_vernuvshiesja_1_den'] = df['ym_vernuvshiesja_1_den'].median()
df.loc[df['ym_vernuvshiesja_2_7_dnej'].isnull(), 'ym_vernuvshiesja_2_7_dnej'] = df['ym_vernuvshiesja_2_7_dnej'].median()
df.loc[df['ym_data_pervogo_vizita'].isnull(), 'ym_data_pervogo_vizita'] = 'undefined'
df.loc[df['ym_dostignutaja_tsel'].isnull(), 'ym_dostignutaja_tsel'] = 'undefined'
df.loc[df['ym_dostignutaja_tsel'] == 'Не определено', 'ym_dostignutaja_tsel'] = 'undefined'
df.loc[df['ym_put_polnyj_stranitsy_vhoda'].isnull(), 'ym_put_polnyj_stranitsy_vhoda'] = 'undefined'
df.loc[df['ym_put_polnyj_stranitsy_vyhoda'].isnull(), 'ym_put_polnyj_stranitsy_vyhoda'] = 'undefined'
df.loc[df['ym_mobilnost'].isnull(), 'ym_mobilnost'] = 'undefined' # Если не определено
df.loc[df['ym_mobilnost'] == 1.0, 'ym_mobilnost'] = 1
df.loc[df['ym_mobilnost'] == 0.0, 'ym_mobilnost'] = 0

In [17]:
# Приведение типов
df = dp.astype_col(df, ['ym_vizity'], coltype='uint8')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58888 entries, 0 to 58887
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ym_clientid                     58888 non-null  object 
 1   ym_pervyj_istochnik_trafika     58888 non-null  object 
 2   ym_pervyj_perehod_s_sajtov      5754 non-null   object 
 3   ym_pervaja_poiskovaja_sistema   3972 non-null   object 
 4   ym_utm_source                   17503 non-null  object 
 5   ym_utm_content                  17324 non-null  object 
 6   ym_data_vizita                  58888 non-null  object 
 7   ym_vizity                       58888 non-null  uint8  
 8   ym_otkazy                       58888 non-null  float64
 9   ym_glubina_prosmotra            58888 non-null  float64
 10  ym_vremja_na_sajte              58888 non-null  object 
 11  ym_novye_posetiteli             58888 non-null  object 
 12  ym_dnej_mezhdu_vizitami         

In [19]:
# Просмотр названий каждого канала
df['ym_pervyj_istochnik_trafika'].unique()

array(['Переходы по рекламе', 'Прямые заходы', 'Внутренние переходы',
       'Переходы из поисковых систем', 'Переходы по ссылкам на сайтах',
       'Переходы из мессенджеров', 'Переходы из рекомендательных систем',
       'Переходы с почтовых рассылок', 'Переходы из социальных сетей',
       'undefined', 'Переходы с сохранённых страниц'], dtype=object)

In [20]:
# Добавляем стоимость клика по каждому каналу
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Прямые заходы', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по рекламе', 'ym_cost'] = 15
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из поисковых систем', 'ym_cost'] = 48.5
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Внутренние переходы', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по ссылкам на сайтах', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из социальных сетей', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с почтовых рассылок', 'ym_cost'] = 2
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из рекомендательных систем', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из мессенджеров', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с сохранённых страниц', 'ym_cost'] = 0
df.loc[df['ym_pervyj_istochnik_trafika'].isnull(), 'ym_pervyj_istochnik_trafika'] = 'undefined'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'undefined', 'ym_cost'] = 0

In [21]:
# Переименовываем категории каналов
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Прямые заходы', 'ym_pervyj_istochnik_trafika'] = 'Browser-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по рекламе', 'ym_pervyj_istochnik_trafika'] = 'Ad-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из поисковых систем', 'ym_pervyj_istochnik_trafika'] = 'Search'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Внутренние переходы', 'ym_pervyj_istochnik_trafika'] = 'Site-Traffic'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы по ссылкам на сайтах', 'ym_pervyj_istochnik_trafika'] = 'Sites'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из социальных сетей', 'ym_pervyj_istochnik_trafika'] = 'Social'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с почтовых рассылок', 'ym_pervyj_istochnik_trafika'] = 'Email'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из рекомендательных систем', 'ym_pervyj_istochnik_trafika'] = 'RecSys'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы из мессенджеров', 'ym_pervyj_istochnik_trafika'] = 'Messenger'
df.loc[df['ym_pervyj_istochnik_trafika'] == 'Переходы с сохранённых страниц', 'ym_pervyj_istochnik_trafika'] = 'SavedPages'

In [22]:
df.head(5)

,ym_clientid,ym_pervyj_istochnik_trafika,ym_pervyj_perehod_s_sajtov,ym_pervaja_poiskovaja_sistema,ym_utm_source,ym_utm_content,ym_data_vizita,ym_vizity,ym_otkazy,ym_glubina_prosmotra,...,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli,ym_data_pervogo_vizita,ym_gorod,ym_dostignutaja_tsel,ym_put_polnyj_stranitsy_vhoda,ym_put_polnyj_stranitsy_vyhoda,ym_mobilnost,ym_cost
0,15785624736522097,Ad-Traffic,NaN,NaN,NaN,NaN,2020-01-11,15,0.066667,3.800000,...,0.933333,0.066667,57,undefined,undefined,undefined,undefined,undefined,undefined,15.0
1,158240803283368197,Browser-Traffic,NaN,NaN,NaN,NaN,2020-02-23,15,1.000000,1.000000,...,0.000000,0.000000,4,undefined,undefined,undefined,undefined,undefined,undefined,0.0
2,1594782414605357519,Ad-Traffic,NaN,NaN,NaN,NaN,2020-07-21,14,1.000000,1.000000,...,0.928571,0.071429,0,undefined,undefined,undefined,undefined,undefined,undefined,15.0
3,1595636421722944149,Ad-Traffic,NaN,NaN,NaN,NaN,2020-07-25,14,0.571429,1.928571,...,0.928571,0.000000,0,2020-07-25,Нур-Султан,undefined,/#secondPage,/#secondPage,1,15.0
4,1600595766306691164,Ad-Traffic,NaN,NaN,yad,300x300,2020-09-20,11,0.181818,1.000000,...,0.909091,0.000000,0,undefined,undefined,undefined,undefined,undefined,undefined,15.0


In [23]:
# Слияние столбцов
df['ym_source'] = df[df.columns[2:6]].apply(lambda x: ' > '.join(x.dropna().astype(str)), axis=1)

In [24]:
# Выводим имена столцов
dp.get_idx_col(df.columns)

0 ym_clientid
1 ym_pervyj_istochnik_trafika
2 ym_pervyj_perehod_s_sajtov
3 ym_pervaja_poiskovaja_sistema
4 ym_utm_source
5 ym_utm_content
6 ym_data_vizita
7 ym_vizity
8 ym_otkazy
9 ym_glubina_prosmotra
10 ym_vremja_na_sajte
11 ym_novye_posetiteli
12 ym_dnej_mezhdu_vizitami
13 ym_vernuvshiesja_1_den
14 ym_vernuvshiesja_2_7_dnej
15 ym_dostizhenija_ljuboj_tseli
16 ym_data_pervogo_vizita
17 ym_gorod
18 ym_dostignutaja_tsel
19 ym_put_polnyj_stranitsy_vhoda
20 ym_put_polnyj_stranitsy_vyhoda
21 ym_mobilnost
22 ym_cost
23 ym_source
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [25]:
# Новый порядок столбцов
new_order = [0, 1, 23, 2, 3, 4, 5, 17, 6, 22, 16, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21]
df = df[df.columns[new_order]]

In [26]:
df.columns

Index(['ym_clientid', 'ym_pervyj_istochnik_trafika', 'ym_source',
       'ym_pervyj_perehod_s_sajtov', 'ym_pervaja_poiskovaja_sistema',
       'ym_utm_source', 'ym_utm_content', 'ym_gorod', 'ym_data_vizita',
       'ym_cost', 'ym_data_pervogo_vizita', 'ym_vizity', 'ym_otkazy',
       'ym_glubina_prosmotra', 'ym_vremja_na_sajte', 'ym_novye_posetiteli',
       'ym_dnej_mezhdu_vizitami', 'ym_vernuvshiesja_1_den',
       'ym_vernuvshiesja_2_7_dnej', 'ym_dostizhenija_ljuboj_tseli',
       'ym_dostignutaja_tsel', 'ym_put_polnyj_stranitsy_vhoda',
       'ym_put_polnyj_stranitsy_vyhoda', 'ym_mobilnost'],
      dtype='object')

In [27]:
# Корректировка значений
df['ym_source'] = df['ym_source'].str.lower() # Переводим все нижний регистр
df.loc[(df['ym_source'] == 'яндекс') | (df['ym_source'] == 'yandex.ru'), 'ym_source'] = 'yandex'
df.loc[(df['ym_source'] == '') | (df['ym_source'].isnull()), 'ym_source'] = 'undefined' # Убираем Null и пустоты
df = df[~df['ym_source'].str.contains('([0-9]{1,3}[\.]){3}[0-9]{1,3}')] # Убираем ip-адреса
df.loc[df['ym_source'] == 'мобильные%сотрудники', 'ym_source'] = 'undefined'
df.loc[df['ym_source'] == 'старт за 2 дня', 'ym_source'] = 'undefined'
df.loc[df['ym_source'] == 'яндекс > serp', 'ym_source'] = 'yandex'
# -------
# Не удаляем, они нужны для ML - модели
df.loc[df['ym_pervyj_perehod_s_sajtov'].isnull(), 'ym_pervyj_perehod_s_sajtov'] = 'undefined'
df.loc[df['ym_pervaja_poiskovaja_sistema'].isnull(), 'ym_pervaja_poiskovaja_sistema'] = 'undefined'
df.loc[df['ym_utm_source'].isnull(), 'ym_utm_source'] = 'undefined'
df.loc[df['ym_utm_content'].isnull(), 'ym_utm_content'] = 'undefined'

In [28]:
df.head(10)

,ym_clientid,ym_pervyj_istochnik_trafika,ym_source,ym_pervyj_perehod_s_sajtov,ym_pervaja_poiskovaja_sistema,ym_utm_source,ym_utm_content,ym_gorod,ym_data_vizita,ym_cost,...,ym_vremja_na_sajte,ym_novye_posetiteli,ym_dnej_mezhdu_vizitami,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli,ym_dostignutaja_tsel,ym_put_polnyj_stranitsy_vhoda,ym_put_polnyj_stranitsy_vyhoda,ym_mobilnost
0,15785624736522097,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,undefined,2020-01-11,15.0,...,00:21:41,0,0.000000,0.933333,0.066667,57,undefined,undefined,undefined,undefined
1,158240803283368197,Browser-Traffic,undefined,undefined,undefined,undefined,undefined,undefined,2020-02-23,0.0,...,00:00:00,15,0.000000,0.000000,0.000000,4,undefined,undefined,undefined,undefined
2,1594782414605357519,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,undefined,2020-07-21,15.0,...,00:00:00,0,0.357143,0.928571,0.071429,0,undefined,undefined,undefined,undefined
3,1595636421722944149,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,Нур-Султан,2020-07-25,15.0,...,00:00:00,1,0.000000,0.928571,0.000000,0,undefined,/#secondPage,/#secondPage,1
4,1600595766306691164,Ad-Traffic,yad > 300x300,undefined,undefined,yad,300x300,undefined,2020-09-20,15.0,...,00:00:10,1,0.000000,0.909091,0.000000,0,undefined,undefined,undefined,undefined
5,1603390261565159009,Ad-Traffic,yad > 240x400,undefined,undefined,yad,240x400,undefined,2020-10-26,15.0,...,00:00:13,0,0.454545,0.909091,0.090909,0,undefined,undefined,undefined,undefined
6,1544792334967355948,Browser-Traffic,undefined,undefined,undefined,undefined,undefined,undefined,2020-06-26,0.0,...,00:06:52,0,0.000000,0.900000,0.100000,0,undefined,undefined,undefined,undefined
7,1580315607562323972,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,undefined,2020-01-29,15.0,...,00:00:00,1,0.000000,0.900000,0.000000,0,undefined,undefined,undefined,undefined
8,15811969981069139018,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,Майкоп,2020-02-09,15.0,...,00:00:02,1,0.000000,0.900000,0.000000,0,undefined,/,/,1
9,159789595254507935,Ad-Traffic,undefined,undefined,undefined,undefined,undefined,undefined,2020-08-20,15.0,...,00:00:11,1,0.000000,0.888889,0.000000,0,undefined,/,/,1


In [29]:
df.describe()

,ym_cost,ym_vizity,ym_otkazy,ym_glubina_prosmotra,ym_dnej_mezhdu_vizitami,ym_vernuvshiesja_1_den,ym_vernuvshiesja_2_7_dnej,ym_dostizhenija_ljuboj_tseli
count,58882.000000,58882.000000,58882.000000,58882.000000,58882.000000,58882.000000,58882.000000,58882.000000
mean,14.705649,1.137835,0.436019,1.337200,0.308058,0.056834,0.016726,0.917989
std,10.673279,0.487498,0.487770,1.200043,3.158002,0.169776,0.112056,2.242552
min,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,15.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,15.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,15.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,48.500000,15.000000,1.000000,63.000000,97.000000,1.000000,1.000000,125.000000


In [30]:
df.shape

(58882, 24)

## Сохранение данных

In [31]:
# Сохранение в файл
df.to_csv('../data/td_metrika.csv', index=False, encoding='utf-8', sep=';')